In [1]:
import time
import gym
import numpy as np
from gym import spaces
from tqdm import tqdm


import numpy as np
from utils.skeleton import *
from utils.quaternion import *
from utils.blazepose import blazepose_skeletons
import os
from pypot.creatures import PoppyTorso
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
#from pypot.creatures.ik import IKChain
from pypot.primitive.move import Move
from pypot.primitive.move import MovePlayer

import sys
sys.path.append('E:\Anaconda\envs\gym-examples')
import gym_examples
from gym.wrappers import FlattenObservation

In [2]:
env = gym.make('gym_examples/Poppy-v0')
env.reset()

pygame 2.1.0 (SDL 2.0.16, Python 3.10.9)
Hello from the pygame community. https://www.pygame.org/contribute.html
Hello, I am Poppy!


array([ 0.1087558 , -0.17607144,  0.07120024, -0.10220377, -0.18020962,
        0.07149935], dtype=float32)

In [3]:
# from stable_baselines3.common.env_checker import check_env
# check_env(env)

In [ ]:
from stable_baselines3 import DDPG
from stable_baselines3.common.noise import NormalActionNoise, OrnsteinUhlenbeckActionNoise

#env = gym.make("Pendulum-v1")

# The noise objects for DDPG
n_actions = env.action_space.shape[-1]
action_noise = NormalActionNoise(mean=np.zeros(n_actions), sigma=0.1 * np.ones(n_actions))

model = DDPG("MlpPolicy", env, action_noise=action_noise, verbose=1)
model.learn(total_timesteps= 3, log_interval=10)
model.save("ddpg_imitation")
vec_env = model.get_env()

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
current step :  0
reward :  -0.45739814487807495
episode :  0
current step :  1
reward :  -0.3705151361760913
episode :  0
current step :  2
reward :  -0.5428800189170493
episode :  0
current step :  3
reward :  -0.48326996615780254
episode :  0
current step :  4
reward :  -0.5136419496943628
episode :  0
current step :  5
reward :  -0.48219605538799826
episode :  0
current step :  6
reward :  -0.4837612050813444
episode :  0
current step :  7
reward :  -0.5212387739713105
episode :  0
current step :  8
reward :  -0.5040817267809566
episode :  0
current step :  9
reward :  -0.5274291257446315
episode :  0
current step :  10
reward :  -0.6322599748836248
episode :  0
current step :  11
reward :  -0.6009901149209714
episode :  0
current step :  12
reward :  -0.6089002628011264
episode :  0
current step :  13
reward :  -0.6673896614735259
episode :  0
current step :  14
reward :  -0.3783929341429

current step :  130
reward :  -0.4574035210295705
episode :  0
current step :  131
reward :  -0.4984373840656076
episode :  0
current step :  132
reward :  -0.39185757012182254
episode :  0
current step :  133
reward :  -0.4208950922854859
episode :  0
current step :  134
reward :  -0.4055671184230996
episode :  0
current step :  135
reward :  -0.32294673321945777
episode :  0
current step :  136
reward :  -0.32120024757989674
episode :  0
current step :  137
reward :  -0.45519060815320334
episode :  0
current step :  138
reward :  -0.35894377177120185
episode :  0
current step :  139
reward :  -0.46268888320985097
episode :  0
current step :  140
reward :  -0.3674995380374069
episode :  0
current step :  141
reward :  -0.4980969724622929
episode :  0
current step :  142
reward :  -0.4259862988520666
episode :  0
current step :  143
reward :  -0.37736954642635884
episode :  0
current step :  144
reward :  -0.4198813874316939
episode :  0
current step :  145
reward :  -0.475058413851284

For training, we can stop here. We can begin the test.

In [ ]:
del model # remove to demonstrate saving and loading

model = DDPG.load("ddpg_imitation")

obs = vec_env.reset()

for t in env.targets:
    action, _states = model.predict(np.array(t.flatten()).reshape(1,-1))
    obs, rewards, dones, info = vec_env.step(action)

The below is just some other tials, you can ignore it.

In [ ]:
obs = vec_env.reset()
# while True:
#     action, _states = model.predict(obs)
#     obs, rewards, dones, info = vec_env.step(action)

for t in env.targets:
    action, _states = model.predict(np.array(t.flatten()).reshape(1,-1))
    obs, rewards, dones, info = vec_env.step(action)

In [ ]:
from stable_baselines3 import DDPG
from stable_baselines3.common.noise import NormalActionNoise, OrnsteinUhlenbeckActionNoise

#env = gym.make("Pendulum-v1")

# The noise objects for DDPG
n_actions = env.action_space.shape[-1]
action_noise = NormalActionNoise(mean=np.zeros(n_actions), sigma=0.1 * np.ones(n_actions))

model = DDPG("MlpPolicy", env, action_noise=action_noise, verbose=1)

model = DDPG.load("ddpg_imitation")
model.get_env()

In [ ]:

for t in env.targets:
    action, _states = model.predict(np.array(t.flatten()).reshape(1,-1))
    obs, rewards, dones, info = env.step(action.flatten())